In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<font size="6"> (BikeShare_Toronto_Ridership_Data_2017-2020.)</font>

 [BikeShare Toronto Ridership Data 2017-2020](https://www.kaggle.com/jasonzxho/bikeshare-toronto-ridership-data-20172020)
 
<font size="6"> by (Peter Gamal Girgis)</font>

<span style="color:orange"><font size="4"> **Task Details:** </font></span>

><font size="5"> About the Dataset</font>
>
>The following dataset contain bike trips taken during the year 2017-2020 across more than 600 stations across the City of Toronto. The CSVs for 2017 and 2018 contains 9 features. The description of each feature is listed below:
>
>>    `trip_id`: Unique ID code for individual trip taken.
>>
>>    `trip_start_time`: Trip start time.
>>
>>    `trip_end_time`: Trip end time.
>>
>>    `trip_duration_seconds`: Duration of the trip in seconds.
>>
>>    `from_station_id`: Unique ID code for the start station.
>>
>>    `from_station_name`: Name of start station.
>>
>>    `to_station_id`: Unique ID code for the end station.
>>
>>    `to_station_name`: Name of end station.
>>
>>    `user_type`: Type of user, either Member or Casual.
>>
>
> The CSVs for 2019 and 2020 contains 11 features. The description of each feature is listed below:
>
>>    `Trip Id`: Unique ID code for individual trip taken.
>>    Subscription Id: Unique ID code for the individual member, this can be used to track Annual Member usage.
>>
>>    `Trip Duration`: Duration of the trip in seconds.
>>
>>    `Start Station Id`: Unique ID code for the start station.
>>
>>    `Start Time`: Trip start time.
>>
>>    `Start Station Name`: Name of start station.
>>
>>    `End Station Id`: Unique ID code for the end station.
>>
>>    `End Time`: Trip end time.
>>
>>    `End Station Name`: Name of end station.
>>
>>   `Bike Id`: Unique ID for the individual bike used.
>> 
>>   `User Type`: Type of user, either Annual or Casual.
 

In [1]:
# Import all Requiring Libraries
import os
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from datetime import timedelta

# Combining several CSV files
from glob import glob 

# to get web contents
import json
from urllib.request import urlopen

import holidays

import pycountry
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from collections import namedtuple

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

# hide warnings
import warnings
warnings.simplefilter('ignore')

<span style="color:orange"><font size="4"> **1. Year_2017** </font></span>

In [1]:
# Using glob function to combine several CSV files with same strucure at 2017 
files = sorted (glob('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2017/2017-Q*.csv'))
files

In [1]:
# Using concat to combined all files and assign() methods
Y_2017 = pd.concat((pd.read_csv(file).assign(filename = file) for file in files), ignore_index = True)
Y_2017

In [1]:
# check unique file's names
Y_2017.filename.unique()

In [1]:
# Modify file path to Y_2017 number.
Y_2017.filename = Y_2017.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2017/', '')
Y_2017.filename = Y_2017.filename.str.replace('.csv', '')

Y_2017.head()

In [1]:
# Change column "filename to "Quarter"
Y_2017 = Y_2017.rename(columns = {'filename' :'Quarter'})
Y_2017.head()

In [1]:
# Check Dataframe shape
Y_2017.shape

In [1]:
# Check Dataframe info()
Y_2017.info()

In [1]:
# check Dataframe NULL values
Y_2017.isnull().sum()

In [1]:
# Found "NULLNULL" values
Y_2017.query('trip_stop_time == "NULLNULL"')

> Cancel trip while `trip_duration_seconds` & `trip_stop_time` equal Zero and no distination at `to_station_name` or `to_station_id`

In [1]:
# Drop Row's have NULLNULL Values
Y_2017 = Y_2017[Y_2017['trip_stop_time'] != 'NULLNULL']
Y_2017

In [1]:
# Check "NULLNULL" values are droped
Y_2017.query('trip_stop_time == "NULLNULL"')

In [1]:
# Check Dataframe NULL values
Y_2017.isnull().sum()

In [1]:
# Select certian columns and add it at new DataFrame "Y_17"
Y_17 = Y_2017[['from_station_id','from_station_name']]
Y_17 = Y_17.rename(columns = {'from_station_id':'station_id','from_station_name':'station_name'})
Y_17

In [1]:
# Select certian columns and add it at new DataFrame "Z_17"
Z_17 = Y_2017[['to_station_id','to_station_name']]
Z_17 = Z_17.rename(columns = {'to_station_id':'station_id','to_station_name':'station_name'})
Z_17

In [1]:
# Concatenate new Dtaframes "Y_17 & Z_17" at new Dtaframe "X_17"
X_17 = pd.concat([Y_17,Z_17])
X_17

In [1]:
# Check number of Unique values
X_17.station_id.nunique()

In [1]:
# Check Dataframe duplication
X_17.duplicated().sum()

In [1]:
# Drop douplication at X_17
X_17 = X_17.drop_duplicates()
X_17

In [1]:
# Drop Row's have NULL Values in column "station_id"
X_17 = X_17[pd.notnull(X_17['station_id'])]
X_17

In [1]:
# Merge 2 Dataframe "X_17" and "Y_2017"
Y_2017 = Y_2017.merge(X_17, left_on = 'from_station_name', right_on = 'station_name', how = 'left')
Y_2017

In [1]:
# Replace NUll Value in column "from_station_id" with data at column "station_id"
Y_2017['from_station_id'] = Y_2017.from_station_id.fillna(Y_2017.station_id)

# Drop un-necessary columns
Y_2017 = Y_2017.drop(['station_id','station_name'], axis = 1)

# check Dataframe NULL values
Y_2017.isnull().sum()

In [1]:
# merge 2 Dataframe "X_17" and "Y_2017"
Y_2017 = Y_2017.merge(X_17, left_on = 'to_station_name', right_on = 'station_name', how = 'left')
Y_2017

In [1]:
# Replace NUll Value in column "to_station_id" with data at column "station_id"
Y_2017['to_station_id'] = Y_2017.to_station_id.fillna(Y_2017.station_id)

# Drop un-necessary columns
Y_2017 = Y_2017.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_2017.isnull().sum()

In [1]:
# save "Y_2017" dataframe in CSV format at new created folder(gathering_data)
Y_2017.to_csv('/kaggle/working/Y_2017.csv', index=False)

<span style="color:orange"><font size="4"> **2. Year_2018** </font></span>

In [1]:
## Using glob function to combine several CSV files with same strucure at 2018 
files = sorted (glob('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2018/2018-Q*.csv'))
files

In [1]:
# Using concat to combined all files and assign() methods
Y_2018 = pd.concat((pd.read_csv(file).assign(filename = file) for file in files), ignore_index = True)
Y_2018

In [1]:
# Check unique file's names
Y_2018.filename.unique()

In [1]:
# Modify file path to Y_2018 number.
Y_2018.filename = Y_2018.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2018/', '')
Y_2018.filename = Y_2018.filename.str.replace('.csv', '')

Y_2018.head()

In [1]:
# Change column "filename to "Quarter"
Y_2018 = Y_2018.rename(columns = {'filename' :'Quarter'})
Y_2018.head()

In [1]:
# Check Dataframe shape
Y_2018.shape

In [1]:
# Check Dataframe info()
Y_2018.info()

In [1]:
# Select certian columns and add it at new DataFrame "Y_18"
Y_18 = Y_2018[['from_station_id','from_station_name']]
Y_18 = Y_18.rename(columns = {'from_station_id':'station_id','from_station_name':'station_name'})
Y_18

In [1]:
# Select certian columns and add it at new DataFrame "Z_18"
Z_18 = Y_2018[['to_station_id','to_station_name']]
Z_18 = Z_18.rename(columns = {'to_station_id':'station_id','to_station_name':'station_name'})
Z_18

In [1]:
# Concatenate new Dtaframes "Y_18 & Z_18" at new Dtaframe "X_18"
X_18 = pd.concat([Y_18,Z_18])
X_18

In [1]:
# Check number of Unique values
X_18.station_id.nunique()

In [1]:
# check Dataframe duplication
X_18.duplicated().sum()

In [1]:
# Drop douplication at X_18
X_18 = X_18.drop_duplicates()
X_18

In [1]:
# Drop Row's have NULL Values
X_18 = X_18.dropna(axis = 0)
X_18

In [1]:
# Check Dataframe NULL values
Y_2018.isnull().sum()

In [1]:
# Check Dataframe duplication
Y_2018.duplicated().sum()

In [1]:
# save "Y_2018" dataframe in CSV format at new created folder(gathering_data)
Y_2018.to_csv('/kaggle/working/Y_2018.csv', index=False)

<span style="color:orange"><font size="4"> **3. Year_2019** </font></span>

In [1]:
## Using glob function to combine several CSV files with same strucure at 2019 
files = sorted (glob('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2019/2019-Q*.csv'))
files

In [1]:
# Using concat to combined all files and assign() methods
Y_2019 = pd.concat((pd.read_csv(file).assign(filename = file) for file in files), ignore_index = True)
Y_2019

In [1]:
# Check unique file's names
Y_2019.filename.unique()

In [1]:
# Modify file path to Y_2019 number.
Y_2019.filename = Y_2019.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2019/', '')
Y_2019.filename = Y_2019.filename.str.replace('.csv', '')

Y_2019.head()

In [1]:
# Change column "filename to "Quarter"
Y_2019 = Y_2019.rename(columns = {'filename' :'Quarter'})
Y_2019.head()

In [1]:
# Check Dataframe shape
Y_2019.shape

In [1]:
# Check Dataframe info()
Y_2019.info()

In [1]:
# Rename columns to matching with columns at other years.
Y_2019 = Y_2019.rename(columns = {'Trip Id':'trip_id', 'Trip  Duration':'trip_duration_seconds', 'Start Station Id':'from_station_id',
                                  'Start Time':'trip_start_time', 'Start Station Name':'from_station_name', 'End Station Id':'to_station_id',
                                 'End Time':'trip_stop_time', 'End Station Name':'to_station_name', 'User Type': 'user_type'})

In [1]:
# Drop un-necessary column
Y_2019 = Y_2019.drop(['Subscription Id','Bike Id'], axis = 1)
Y_2019.info()

In [1]:
# Check Dataframe NULL values
Y_2019.isnull().sum()

In [1]:
# Select certian columns and add it at new DataFrame "Y_19"
Y_19 = Y_2019[['from_station_id','from_station_name']]
Y_19 = Y_19.rename(columns = {'from_station_id':'station_id','from_station_name':'station_name'})
Y_19

In [1]:
# Select certian columns and add it at new DataFrame "Z_19"
Z_19 = Y_2019[['to_station_id','to_station_name']]
Z_19 = Z_19.rename(columns = {'to_station_id':'station_id','to_station_name':'station_name'})
Z_19

In [1]:
# Concatenate new Dtaframes "Y_19 & Z_19" at new Dtaframe "X_19"
X_19 = pd.concat([Y_19,Z_19])
X_19

In [1]:
# Check number of Unique values
X_19.station_id.nunique()

In [1]:
# Check Dataframe duplication
X_19.duplicated().sum()

In [1]:
# Drop douplication at X_19
X_19 = X_19.drop_duplicates()
X_19

In [1]:
# Drop Row's have NULL Values related column "station_name"
X_19 = X_19[pd.notnull(X_19['station_name'])]
X_19

In [1]:
# Check Dataframe NULL values
Y_2019.isnull().sum()

In [1]:
# Merge 2 Dataframe "X_19" and "Y_2019"
Y_2019 = Y_2019.merge(X_19, left_on = 'to_station_name', right_on = 'station_name', how = 'left')
Y_2019

In [1]:
# Replace NUll Value in column "to_station_id" with data at column "station_id"
# Replace NUll Value in column "to_station_name" with data at column "station_name"
Y_2019['to_station_id'] = Y_2019.to_station_id.fillna(Y_2019.station_id)
Y_2019['to_station_name'] = Y_2019.to_station_name.fillna(Y_2019.station_name)

# Drop un-necessary column
Y_2019 = Y_2019.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_2019.isnull().sum()

In [1]:
# Check Dataframe duplication
Y_2019.duplicated().sum()

In [1]:
# Save "Y_2019" dataframe in CSV format at new created folder(gathering_data)
Y_2019.to_csv('/kaggle/working/Y_2019.csv', index=False)

<span style="color:orange"><font size="4"> **4. Year_2020** </font></span>

In [1]:
## Using glob function to combine several CSV files with same strucure at 2020 
files = sorted (glob('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-*.csv'))
files

In [1]:
# Using concat to combined all files and assign() methods
Y_2020 = pd.concat((pd.read_csv(file).assign(filename = file) for file in files), ignore_index = True)
Y_2020.head()

In [1]:
# Check unique file's names
Y_2020.filename.unique()

In [1]:
# Modify file path to Y_2020 number.
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-01','2020-Q1')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-02','2020-Q1')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-03','2020-Q1')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-04','2020-Q2')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-05','2020-Q2')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-06','2020-Q2')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-07','2020-Q3')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-08','2020-Q3')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-09','2020-Q3')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-10','2020-Q4')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-11','2020-Q4')
Y_2020.filename = Y_2020.filename.str.replace('/kaggle/input/bikeshare-toronto-ridership-data-20172020/Bikeshare 2020/2020-12','2020-Q4')
Y_2020.filename = Y_2020.filename.str.replace('.csv', '')

Y_2020.head()

In [1]:
# Change column "filename to "Quarter"
Y_2020 = Y_2020.rename(columns = {'filename' :'Quarter'})
Y_2020.head()

In [1]:
# Check Columns names at DataFrame "Y_2020"
Y_2020.columns

In [1]:
# Check Dataframe shape
Y_2020.shape

In [1]:
# Check Dataframe info()
Y_2020.info()

In [1]:
# Rename columns to matching with columns at other years.
Y_2020 = Y_2020.rename(columns = {'Trip Id':'trip_id', 'Trip  Duration':'trip_duration_seconds', 'Start Station Id':'from_station_id',
                                  'Start Time':'trip_start_time', 'Start Station Name':'from_station_name', 'End Station Id':'to_station_id',
                                 'End Time':'trip_stop_time', 'End Station Name':'to_station_name', 'User Type': 'user_type'})

In [1]:
# Check Dataframe info()
Y_2020.info()

In [1]:
# Drop un-necessary column
Y_2020 = Y_2020.drop(['Subscription Id','Bike Id', '10293877', '863410', '905', '7038', '11/01/2020 00:00', 'Dundas St W / Yonge St',
                     '7253', '11/01/2020 00:15', 'John St  / Mercer St - SMART', '2260', 'Casual Member', '10529965', '832983', '304', '7542',
                     '12/01/2020 00:02', 'Queen St W / John St', '7544', '12/01/2020 00:07', 'Foster Pl / Elizabeth St - SMART', '1182',
                     'Annual Member'], axis = 1)
Y_2020.info()

In [1]:
# Check Dataframe NULL values
Y_2020.isnull().sum()

In [1]:
# Drop Row's have NULL Values related column "trip_id"
Y_2020 = Y_2020[pd.notnull(Y_2020['trip_id'])]
Y_2020

In [1]:
# Check Dataframe NULL values
Y_2020.isnull().sum()

In [1]:
# Select certian columns and add it at new DataFrame "Y_20"
Y_20 = Y_2020[['from_station_id','from_station_name']]
Y_20 = Y_20.rename(columns = {'from_station_id':'station_id','from_station_name':'station_name'})
Y_20

In [1]:
# Select certian columns and add it at new DataFrame "Z_20"
Z_20 = Y_2020[['to_station_id','to_station_name']]
Z_20 = Z_20.rename(columns = {'to_station_id':'station_id','to_station_name':'station_name'})
Z_20

In [1]:
# Concatenate new Dtaframes "Y_20 & Z_20" at new Dtaframe "X_20"
X_20 = pd.concat([Y_20,Z_20])
X_20

In [1]:
# Check Dataframe duplication
X_20.duplicated().sum()

In [1]:
# Drop douplication at X_20
X_20 = X_20.drop_duplicates()
X_20

In [1]:
# Drop Row's have NULL Values related column "station_name"
X_20 = X_20[pd.notnull(X_20['station_name'])]
X_20

In [1]:
# Merge 2 Dataframe "X_20" and "Y_2020"
Y_2020 = Y_2020.merge(X_20, left_on = 'from_station_id', right_on = 'station_id', how = 'left')
Y_2020

In [1]:
# Replace NUll Value in column "from_station_name" with data at column "station_name"
Y_2020['from_station_name'] = Y_2020.from_station_name.fillna(Y_2020.station_name)

# Drop un-necessary column
Y_2020 = Y_2020.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_2020.isnull().sum()

In [1]:
# Merge 2 Dataframe "X_20" and "Y_2020"
Y_2020 = Y_2020.merge(X_20, left_on = 'to_station_id', right_on = 'station_id', how = 'left')
Y_2020

In [1]:
# Replace NUll Value in column "to_station_name" with data at column "station_name"
Y_2020['to_station_name'] = Y_2020.to_station_name.fillna(Y_2020.station_name)

# Drop un-necessary column
Y_2020 = Y_2020.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_2020.isnull().sum()

In [1]:
# check Dataframe duplication
Y_2020.duplicated().sum()

In [1]:
# Save "Y_2020" dataframe in CSV format at new created folder(gathering_data)
Y_2020.to_csv('/kaggle/working/Y_2020.csv', index=False)

<span style="color:orange"><font size="4"> **5. Combine All DF's** </font></span>

In [1]:
## Using glob function to combine several CSV files with same strucure at gathering_data
files = sorted (glob('/kaggle/working/Y_20*.csv'))
files

In [1]:
# using concat to combined all files
Y_All = pd.concat((pd.read_csv(file) for file in files), ignore_index = True)
Y_All

In [1]:
# Select certian columns and add it at new DataFrame "Y"
Y = Y_All[['from_station_id','from_station_name']]
Y = Y.rename(columns = {'from_station_id':'station_id','from_station_name':'station_name'})
Y

In [1]:
# Select certian columns and add it at new DataFrame "Z"
Z = Y_All[['to_station_id','to_station_name']]
Z = Z.rename(columns = {'to_station_id':'station_id','to_station_name':'station_name'})
Z

In [1]:
# Concatenate new Dtaframes "Y & Z" at new Dtaframe "X"
X = pd.concat([Y,Z])
X

In [1]:
# Check Dataframe duplication
X.duplicated().sum()

In [1]:
# Drop douplication at X
X = X.drop_duplicates()
X

In [1]:
# Check NULL values at Column "station_id"
X.query('station_id.isnull()', engine='python')

In [1]:
# Update the repeated stations with same ID number
# Update station_id = 7183 for station "Margueretta St / College St"
X['station_id'][465831] = 7183.0

# Update station_id = 7180 for station "Lansdowne Subway Green P"
X['station_id'][465850] = 7180.0

# Update station_id = 7161 for station "Beverly St / College St"
X['station_id'][465532] = 7161.0

In [1]:
# Update station without ID by adding ID for them
# Station "Michael Sweet Ave / St. Patrick St" using station_id = 7080.0
X['station_id'][465490] = 7080.0

# Station "Roxton Rd / College St" using station_id = 7081.0
X['station_id'][728511] = 7081.0

# Station "Base Station" using station_id = 7082.0
X['station_id'][731204] = 7082.0

# Station "Lake Shore Blvd W / Ontario Dr(Ontario Place)" using station_id = 7212.0
X['station_id'][734637] = 7212.0

# Station "Dovercourt Rd / Harrison St - SMART" using station_id = 7213.0
X['station_id'][797267] = 7213.0

# Station "Summerhill Ave / MacLennan Ave - SMART" using station_id = 7214.0
X['station_id'][800483] = 7214.0

# Station "Fringe Next Stage - 7219" using station_id = 7215.0
X['station_id'][1038094] = 7215.0

In [1]:
# Check above stations name related column"station_name"
X.query('station_name == "Fringe Next Stage - 7219" ')

In [1]:
# Check NULL values at Column "station_id"
X.query('station_id.isnull()', engine='python')

In [1]:
# Drop Row's have NULL Values at X
X = X.dropna(axis = 0)
X

In [1]:
# Check Dataframe NULL values
Y_All.isnull().sum()

In [1]:
# Merge 2 Dataframe "X" and "Y_All"
Y_All = Y_All.merge(X, left_on = 'from_station_name', right_on = 'station_name', how = 'left')
Y_All

In [1]:
# Replace NUll Value in column "from_station_id" with data at column "station_id"
Y_All['from_station_id'] = Y_All.from_station_id.fillna(Y_All.station_id)

# Drop un-necessary column
Y_All = Y_All.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_All.isnull().sum()

In [1]:
# Merge 2 Dataframe "X" and "Y_All"
Y_All = Y_All.merge(X, left_on = 'to_station_name', right_on = 'station_name', how = 'left')
Y_All

In [1]:
# Replace NUll Value in column "to_station_id" with data at column "station_id"
Y_All['to_station_id'] = Y_All.to_station_id.fillna(Y_All.station_id)

# Drop un-necessary column
Y_All = Y_All.drop(['station_id','station_name'], axis = 1)

# Check Dataframe NULL values
Y_All.isnull().sum()

In [1]:
# Correct data type for "trip_start_time , trip_stop_time" to be time stamp
Y_All['trip_start_time']= pd.to_datetime(Y_All.trip_start_time)
Y_All['trip_stop_time'] = pd.to_datetime(Y_All.trip_stop_time)
Y_All.info()

In [1]:
# check total trip time in seconds for all trips and convert it to integer 
Y_All['trip_duration_seconds'] = (Y_All.trip_stop_time - Y_All.trip_start_time).dt.total_seconds().astype('int64')

# Check Dataframe info()
Y_All.info()

In [1]:
# check trip_duration_seconds = zero values ( that mean trips are cancelled)
Y_All.query('trip_duration_seconds == 0')

In [1]:
# Drop Row's have trip_duration_seconds = zero values 
Y_All = Y_All[(Y_All['trip_duration_seconds'] > 0)]
Y_All

In [1]:
# Check trip_duration_seconds = zero values are all removed
Y_All.query('trip_duration_seconds == 0')

In [1]:
# Check Dataframe NULL values
Y_All.isnull().sum()

In [1]:
# Modify "Casual" to "Casual Member" at "user_type" column
Y_All.user_type = Y_All.user_type.replace('Casual', 'Casual Member')

# Modify "Member" to "Annual Member" at "user_type" column
Y_All.user_type = Y_All.user_type.replace('Member', 'Annual Member')

# Check changes applied
Y_All.user_type.unique()

In [1]:
# Save "Y_All" dataframe in CSV format at new created folder(gathering_data)
Y_All.to_csv('/kaggle/working/Y_All.csv', index=False)

<span style="color:orange"><font size="4"> **6. Copying DataFrame** </font></span>

In [1]:
# Copying the merged DataFrame
df_All = Y_All.copy()
df_All

<span style="color:orange"><font size="4"> **7. Exploratory Data Analysis ( Analyzing and Visualization)** </font></span>
><span style="color:orange"><font size="3"> **Q1:How Many user type at All Period?** </font></span>

In [1]:
# Plot "user_type" Analysis
plt.figure(figsize=[6, 6])
color_base = sns.color_palette()[0]

ax = sns.countplot(data = df_All, x = 'user_type', color = color_base, order = df_All.user_type.value_counts().index)

for i,j in enumerate (df_All.user_type.value_counts()):
    ax.text(i,100 + df_All.user_type.value_counts()[i], j, weight = "bold", size = 13,va='baseline', ha='center')
    
plt.yticks([0, 1e6, 2e6, 3e6, 4e6, 5e6, 6e6],['0','1M','2M','3M','4M','5M','6M'])
plt.xlabel("User's Type", size = 15)
plt.ylabel("Total User's Count", size = 15)
plt.title("User's Type Distribution", size = 15, weight = 'bold')
plt.show();

><span style="color:orange"><font size="3"> **Q2:How Many User Type / Quarter in each Year ?** </font></span>

In [1]:
# Plot "user_type" Analysis
date_quarter = df_All.groupby(df_All.Quarter)['user_type'].count()

ax = date_quarter.plot(kind ='line',figsize = (20,10), marker = 'o')       
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ['2017-Q1', '2017-Q2', '2017-Q3', '2017-Q4','2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4',
                                                                    '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4','2020-Q1', '2020-Q2', '2020-Q3', '2020-Q4'])
plt.yticks([0, 0.2e6, 0.4e6, 0.6e6, 0.8e6, 1.0e6, 1.2e6, 1.4e6, 1.6e6],['0', '200K', '400K', '600K','800K', '1M', '1.2M', '1.4M', '1.6M'])
plt.xlabel("Date(Year-Quarter)", size = 15)
plt.ylabel('Total Trips', size = 15)
plt.title("User's Type Distribution / Quarter", weight = 'bold', size = 15)

plt.grid(True)
plt.show();

><span style="color:orange"><font size="3"> **Q3:What's Total Trips per year?** </font></span>

In [1]:
date_year = df_All.trip_start_time.groupby(df_All.trip_start_time.dt.year).count()

ax = date_year.plot(kind ='line',figsize = (6,6), marker = 'o')       
plt.title('Trips per Years', weight = 'bold', size = 15)
plt.xlabel("Date(Year's)", size = 15)
plt.ylabel('Total Trips', size = 15)
plt.xticks([2017.0,2018.0, 2019.0, 2020.0], ['2017', '2018', '2019','2020'])
plt.yticks([1.6e6, 1.8e6, 2.0e6, 2.2e6, 2.4e6, 2.6e6], ['1.6M', '1.8M', '2M','2.2M', '2.4M', '2.6M'])
plt.grid(True)
plt.show();

><span style="color:orange"><font size="3"> **Q4:What's the Total Count Trips per Month?** </font></span>

In [1]:
trips_month = df_All.trip_start_time.groupby([df_All.trip_start_time.dt.year, df_All.trip_start_time.dt.month]).count()

ax = trips_month.plot(kind ='line',figsize = (20,8), marker = 'o')       
plt.title('Total Count Trips per Month', weight = 'bold', size = 15)
plt.xlabel("Date(Month-Year)", size = 15)
plt.ylabel('Total Trips', size = 15)
plt.xticks([0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47], ['Jan-17', 'Mar-17', 'Jun-17', 'Sep-17', 'Dec-17', 'Mar-18', 'Jun-18', 'Sep-18', 'Dec-18',
                                                                  'Mar-19', 'Jun-19', 'Sep-19', 'Dec-19', 'Mar-20', 'Jun-20', 'Sep-20', 'Dec-20' ])
plt.yticks([100000, 200000, 300000, 400000, 500000], ['100K', '200K', '300K','400K', '500K'])
plt.grid(True)
plt.show();

><span style="color:orange"><font size="3"> **Q5: What's the Total Count Trips per Quarter as (Annual / Casual) Member?** </font></span>

In [1]:
# Plot "user_type" Analysis
trips_quarter = df_All.groupby(['Quarter','user_type'])['trip_id'].count().reset_index()
trips_quarter.head()

In [1]:
# combine 2 colums "user_type" & "trip_start_time" 
Annual = pd.DataFrame(trips_quarter.query('user_type == "Annual Member"').value_counts()).reset_index()
# combine 2 colums "user_type" & "trip_start_time" 
Casual = pd.DataFrame(trips_quarter.query('user_type == "Casual Member"').value_counts()).reset_index()
Annual

In [1]:
fig,(ax1,ax2) = plt.subplots (1,2,figsize = (20,20))

# figure "a" represent number of Annual Member in user type 
a = sns.barplot (x = Annual.trip_id, y = Annual.Quarter, ax = ax1 , 
                 linewidth = 1 ,alpha = 0.7, palette = 'Blues_r')

# figure "a" represent number of  Casual Member in user type
b = sns.barplot (x = Casual.trip_id, y = Casual.Quarter, ax = ax2 , 
                 linewidth = 1 ,alpha = 0.7, palette = 'Blues_r')

# create loop to count number of Trips in each Annual Member
for i,j in enumerate(Annual.trip_id):
        ax1.text(.07,i+0.15,j,weight = "bold", size = 15)

# create loop to count number of Trips in each Casual Member
for k,l in enumerate(Casual.trip_id):
        ax2.text(.07,k+0.15,l,weight = "bold", size = 15)     
        
a.set_title("Total Count Trips / Quarter as Annual Member" , weight = 'bold', size = 15)
a.set_xlabel('Total Annual Member Count / Quarter', size = 15)
a.set_ylabel("Date(Year-Quarter)", size = 15)


b.set_title("Total Count Trips / Quarter as Casual Member" , weight = 'bold', size = 15)
b.set_xlabel('Total Casual Member Count / Quarter', size = 15)
b.set_ylabel("Date(Year-Quarter)", size = 15)

plt.show();

><span style="color:orange"><font size="3"> **Q6: What's the Total Period Trip / Month?** </font></span>

In [1]:
total_month_trip = pd.DataFrame(df_All.groupby([df_All.trip_start_time.dt.year, df_All.trip_start_time.dt.month])['trip_duration_seconds'].sum())

ax = total_month_trip.plot(kind ='line',figsize = (20,8), marker = 'o')       
plt.title('Max. Period Trip per Month', weight = 'bold', size = 15)
plt.xlabel("Date(Month-Year)", size = 15)
plt.ylabel('Total Trips / Seconds', size = 15)
plt.xticks([0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47], ['Jan-17', 'Mar-17', 'Jun-17', 'Sep-17', 'Dec-17', 'Mar-18', 'Jun-18', 'Sep-18', 'Dec-18',
                                                                  'Mar-19', 'Jun-19', 'Sep-19', 'Dec-19', 'Mar-20', 'Jun-20', 'Sep-20', 'Dec-20' ])
plt.yticks([1e8, 2e8, 3e8, 4e8, 5e8, 6e8, 7e8, 8e8, 9e8], ['100M', '200M', '300M','400M', '500M', '600M', '700M', '800M', '900M'])
plt.grid(True)
plt.legend('')
plt.show();

><span style="color:orange"><font size="3"> **Q7: What's the Min. Period Trip / Month?** </font></span>

In [1]:
min_month_trip = pd.DataFrame(df_All.groupby([df_All.trip_start_time.dt.year, df_All.trip_start_time.dt.month])['trip_duration_seconds'].idxmin())

ax = min_month_trip.plot(kind ='line',figsize = (20,8), marker = 'o')       
plt.title('Max. Period Trip per Month', weight = 'bold', size = 15)
plt.xlabel("Date(Month-Year)", size = 15)
plt.ylabel('Total Trips / Seconds', size = 15)
plt.xticks([0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47], ['Jan-17', 'Mar-17', 'Jun-17', 'Sep-17', 'Dec-17', 'Mar-18', 'Jun-18', 'Sep-18', 'Dec-18',
                                                                  'Mar-19', 'Jun-19', 'Sep-19', 'Dec-19', 'Mar-20', 'Jun-20', 'Sep-20', 'Dec-20' ])
plt.yticks([0, 1e6, 2e6, 3e6, 4e6, 5e6, 6e6, 7e6, 8e6, 9e6], ['0', '1M', '2M', '3M','4M', '5M', '6M', '7M', '8M', '9M'])
plt.grid(True)
plt.legend('')
plt.show();

><span style="color:orange"><font size="3"> **Q8: What's the Max. Period Trip / Month?** </font></span>

In [1]:
max_month_trip = pd.DataFrame(df_All.groupby([df_All.trip_start_time.dt.year, df_All.trip_start_time.dt.month])['trip_duration_seconds'].idxmax())

ax = max_month_trip.plot(kind ='line',figsize = (20,8), marker = 'o')       
plt.title('Max. Period Trip per Month', weight = 'bold', size = 15)
plt.xlabel("Date(Month-Year)", size = 15)
plt.ylabel('Total Trips / Seconds', size = 15)
plt.xticks([0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47], ['Jan-17', 'Mar-17', 'Jun-17', 'Sep-17', 'Dec-17', 'Mar-18', 'Jun-18', 'Sep-18', 'Dec-18',
                                                                  'Mar-19', 'Jun-19', 'Sep-19', 'Dec-19', 'Mar-20', 'Jun-20', 'Sep-20', 'Dec-20' ])
plt.yticks([1e6, 2e6, 3e6, 4e6, 5e6, 6e6, 7e6, 8e6, 9e6], ['1M', '2M', '3M','4M', '5M', '6M', '7M', '8M', '9M'])
plt.grid(True)
plt.legend('')
plt.show();

><span style="color:orange"><font size="3"> **Q9: What's the Average Period Trip / Month?** </font></span>

In [1]:
average_month_trip = pd.DataFrame(df_All.groupby([df_All.trip_start_time.dt.year, df_All.trip_start_time.dt.month])['trip_duration_seconds'].mean())

ax = average_month_trip.plot(kind ='line',figsize = (20,8), marker = 'o')       
plt.title('Max. Period Trip per Month', weight = 'bold', size = 15)
plt.xlabel("Date(Month-Year)", size = 15)
plt.ylabel('Total Trips / Seconds', size = 15)
plt.xticks([0, 2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47], ['Jan-17', 'Mar-17', 'Jun-17', 'Sep-17', 'Dec-17', 'Mar-18', 'Jun-18', 'Sep-18', 'Dec-18',
                                                                  'Mar-19', 'Jun-19', 'Sep-19', 'Dec-19', 'Mar-20', 'Jun-20', 'Sep-20', 'Dec-20' ])
plt.yticks([2e3, 4e3, 6e3, 8e3, 10e3, 12e3, 14e3], ['2K', '4K', '6K','8K', '10K', '12K', '14K'])
plt.grid(True)
plt.legend('')
plt.show();

><span style="color:orange"><font size="3"> **Q10: What's the Top 20 Stations at Start Point?** </font></span>

In [1]:
# combine 2 colums "from_station_id" & "trip_id" then sort stations desending regarding number of trips results
most_Start_id = df_All.groupby('from_station_name')['trip_id'].count().reset_index().sort_values( by = 'trip_id', ascending = False )

In [1]:
fig, ax1 = plt.subplots (1,figsize = (10,12))

# figure "a" represent number of player heighest
a = sns.barplot (x = most_Start_id.trip_id, y = most_Start_id.from_station_name[:20], ax = ax1 , linewidth = 1 ,alpha = 0.7, palette = 'Blues_r')

# create loop to count players depend on height player.
for i,j in enumerate(most_Start_id.trip_id[:20]):
        ax1.text(.7,i+0.08,j,weight = "bold", size = 15)

a.set_title("Top 20 Stations at start Point Distribution" , weight = 'bold', size = 15)
a.set_xlabel('Total Count', size = 15)
a.set_ylabel("Top Station's Distribution", size = 15)
plt.xticks([0, 2e4, 4e4, 6e4, 8e4, 1e5], ['0', '20K', '40K','60K', '80K', '100K'])
plt.show();

><span style="color:orange"><font size="3"> **Q11: What's the Top 20 Stations at End Point?** </font></span>

In [1]:
# combine 2 colums "to_station_name" & "trip_id" then sort stations desending regarding number of trips results
most_End_id = df_All.groupby('to_station_name')['trip_id'].count().reset_index().sort_values( by = 'trip_id', ascending = False )

In [1]:
fig, ax1 = plt.subplots (1,figsize = (10,12))

# figure "a" represent number of player heighest
a = sns.barplot (x = most_End_id.trip_id, y = most_End_id.to_station_name[:20], ax = ax1 , linewidth = 1 ,alpha = 0.7, palette = 'Blues_r')

# create loop to count players depend on height player.
for i,j in enumerate(most_End_id.trip_id[:20]):
        ax1.text(.7,i+0.08,j,weight = "bold", size = 15)

a.set_title("Top 20 Stations at End Point Distribution" , weight = 'bold', size = 15)
a.set_xlabel('Total Count', size = 15)
a.set_ylabel("Top Station's Distribution", size = 15)
plt.xticks([0, 2e4, 4e4, 6e4, 8e4, 1e5], ['0', '20K', '40K','60K', '80K', '100K'])
plt.show();

><span style="color:orange"><font size="3"> **Q12: What's the Q10: What's Correlation Between Variables?** </font></span>

In [1]:
# get correlation in specific columns
correlation = df_All.corr()
plt.figure(figsize=(8,8))

# create heatmap plot
sns.heatmap(correlation,annot=True ,cmap = plt.cm.plasma, linecolor='white', linewidths=2)
plt.title('Correlation Between Variables')
plt.show();

><span style="color:orange"><font size="3"> **Q13: Using "worldcloud" to print Top stations at Start Point?** </font></span>

In [1]:
from wordcloud import WordCloud
# get player counts value acheived gameId > 32000 
top_rate = most_Start_id[most_Start_id.trip_id  > 30000 ]['from_station_name'].value_counts().index

# using bckground" WorldCloud
w_c = WordCloud(background_color="white",scale=2).generate(" ".join(top_rate))
fig = plt.figure(figsize=(15,8))

# plot show in "bilionear style"
plt.imshow(w_c,interpolation="bilinear")
plt.axis("off")
plt.title("Top stations at Start Point")
plt.show();